# Representing text

If we want to solve Natural Language Processing (NLP) tasks with neural networks, we need some way to represent text as tensors. Computers already represent textual characters as numbers that map to fonts on your screen using encodings such as ASCII or UTF-8.

<img alt="Image showing diagram mapping a character to an ASCII and binary representation" src="images/2-represent-text-as-tensor-checkpoint-1.png" align="middle" />

We understand what each letter **represents**, and how all characters come together to form the words of a sentence. However, computers by themselves do not have such an understanding, and a neural network has to learn the meaning during training.

Therefore, we can use different approaches when representing text:
* **Character-level representation**, when we represent text by treating each character as a number. Given that we have $C$ different characters in our text corpus, the word *Hello* would be represented by $5\times C$ tensor. Each letter would correspond to a tensor column in one-hot encoding.
* **Word-level representation**, when we create a **vocabulary** of all words in our text sequence or sentence(s), and then represent each word using one-hot encoding. This approach is somehow better, because each letter by itself does not have much meaning, and thus by using higher-level semantic concepts - words - we simplify the task for the neural network. However, given a large dictionary size, we need to deal with high-dimensional sparse tensors.  For example, if we have a vocabulary size of 10,000 different words.  Then each word would have an one-hot encoding length of 10,000; hence the high-dimensional.

To unify those approaches, we typically call an atomic piece of text **a token**. In some cases tokens can be letters, in other cases - words, or parts of words.

> For example, we can choose to tokenize *indivisible* as `in`-`divis`-`ible`, where the `#` sign represents that the token is a continuation of the previous word. This would allow the root `divis` to always be represented by one token, corresponding to one core meaning.

The process of converting text into a sequence of tokens is called **tokenization**. Next, we need to assign each token to a number, which we can feed into a neural network. This is called **vectorization**, and is normally done by building a token vocabulary.  

Let's start by installing some required Python packages we'll use in this module.

In [ ]:
!pip install -r https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/requirements.txt

# Text classification task

In this module, we will start with a simple text classification task based on **AG_NEWS** sample dataset, which is to classify news headlines into one of 4 categories: _World, Sports, Business and Sci/Tech_. This dataset is built from PyTorch's `torchtext` module, so we can easily access it.

In [2]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Here, `train_dataset` and `test_dataset` contain iterators that return pairs of label (number of class) and text respectively, for example:

In [3]:
next(train_dataset)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

So, let's print out the first 5 new headlines from our dataset: 

In [4]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}\n")

**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.

**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.

**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.

**Sci/Tech** -> Oil prices soar to all-time record, posing new menace to US economy (AFP) AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace

Because datasets are iterators, if we want to use the data multiple times we need to convert it to a list:

In [5]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenization and Vectorization

Now we need to convert text into **numbers** that can be represented as tensors to feed them into a neural network. The first step is to convert text to tokens - **tokenization**. If we use word-level representation, each word would be represented by its own token. We will use build-in tokenizer from `torchtext` module:

In [6]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

We'll use PyTorch's tokenizer to split words and spaces in the first 2 news articles. In our case, we use basic_english for the tokenizer to understand the language structure. This will return a string list of the text and characters.

In [7]:
first_sentence = train_dataset[0][1]
second_sentence = train_dataset[1][1]

f_tokens = tokenizer(first_sentence)
s_tokens = tokenizer(second_sentence)

print(f'\nfirst token list:\n{f_tokens}')
print(f'\nsecond token list:\n{s_tokens}')


first token list:
['wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')', 'reuters', '-', 'short-sellers', ',', 'wall', 'street', "'", 's', 'dwindling\\band', 'of', 'ultra-cynics', ',', 'are', 'seeing', 'green', 'again', '.']

second token list:
['carlyle', 'looks', 'toward', 'commercial', 'aerospace', '(', 'reuters', ')', 'reuters', '-', 'private', 'investment', 'firm', 'carlyle', 'group', ',', '\\which', 'has', 'a', 'reputation', 'for', 'making', 'well-timed', 'and', 'occasionally\\controversial', 'plays', 'in', 'the', 'defense', 'industry', ',', 'has', 'quietly', 'placed\\its', 'bets', 'on', 'another', 'part', 'of', 'the', 'market', '.']


Next, to convert text to numbers, we will need to build a vocabulary of all tokens. We first build the dictionary using the `Counter` object, and then create a `Vocab` object that would help us deal with vectorization:

In [8]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.Vocab(counter, min_freq=1)

To see how each word maps to the vocabulary, we'll loop through each word in the list to lookup it's index number in `vocab`.  Each word or character is displayed with it's corresponding index.  For example, word 'the' appears several times in both sentence and it's unique index in the vocab is the number 3.

In [9]:
word_lookup = [list((vocab[w], w)) for w in f_tokens]
print(f'\nIndex lockup in 1st sentence:\n{word_lookup}')

word_lookup = [list((vocab[w], w)) for w in s_tokens]
print(f'\nIndex lockup in 2nd sentence:\n{word_lookup}')


Index lockup in 1st sentence:
[[432, 'wall'], [426, 'st'], [2, '.'], [1606, 'bears'], [14839, 'claw'], [114, 'back'], [67, 'into'], [3, 'the'], [849, 'black'], [14, '('], [28, 'reuters'], [15, ')'], [28, 'reuters'], [16, '-'], [50726, 'short-sellers'], [4, ','], [432, 'wall'], [375, 'street'], [17, "'"], [10, 's'], [67508, 'dwindling\\band'], [7, 'of'], [52259, 'ultra-cynics'], [4, ','], [43, 'are'], [4010, 'seeing'], [784, 'green'], [326, 'again'], [2, '.']]

Index lockup in 2nd sentence:
[[15875, 'carlyle'], [1073, 'looks'], [855, 'toward'], [1311, 'commercial'], [4251, 'aerospace'], [14, '('], [28, 'reuters'], [15, ')'], [28, 'reuters'], [16, '-'], [930, 'private'], [798, 'investment'], [321, 'firm'], [15875, 'carlyle'], [99, 'group'], [4, ','], [27658, '\\which'], [29, 'has'], [6, 'a'], [4460, 'reputation'], [12, 'for'], [565, 'making'], [52791, 'well-timed'], [9, 'and'], [80618, 'occasionally\\controversial'], [2126, 'plays'], [8, 'in'], [3, 'the'], [526, 'defense'], [242, 'indus

Using vocabulary, we can easily encode our tokenized string into a set of numbers. Let's use the first news article as an example:

In [10]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

def encode(x):
    return [vocab.stoi[s] for s in tokenizer(x)]

vec = encode(first_sentence)
print(vec)

Vocab size of 95812
[432, 426, 2, 1606, 14839, 114, 67, 3, 849, 14, 28, 15, 28, 16, 50726, 4, 432, 375, 17, 10, 67508, 7, 52259, 4, 43, 4010, 784, 326, 2]


In this code, the torchtext `vocab.stoi` dictionary allows us to convert from a string representation into numbers (the name *stoi* stands for "from **s**tring **to** **i**ntegers). To convert the text back from a numeric representation into text, we can use the `vocab.itos` dictionary to perform reverse lookup:

In [11]:
def decode(x):
    return [vocab.itos[i] for i in x]

decode(vec)

['wall',
 'st',
 '.',
 'bears',
 'claw',
 'back',
 'into',
 'the',
 'black',
 '(',
 'reuters',
 ')',
 'reuters',
 '-',
 'short-sellers',
 ',',
 'wall',
 'street',
 "'",
 's',
 'dwindling\\band',
 'of',
 'ultra-cynics',
 ',',
 'are',
 'seeing',
 'green',
 'again',
 '.']

## BiGrams, TriGrams and N-Grams

One limitation of word tokenization is that some words are part of multi word expressions, for example, the word _'hot dog'_ has a completely different meaning than the words 'hot' and 'dog' in other contexts. If we represent words 'hot` and 'dog' always by the same vectors, it can confuse the model.

To address this, **N-gram representations** are sometimes used in document classification, where the frequency of each word, bi-word or tri-word is a useful feature for training classifiers. 
- In **bigram** representation, for example, we will add all _word pairs_ to the vocabulary, in addition to original words. 

To get n-gram representation, we can use `ngrams_iterator` function that will convert the sequence of tokens to the sequence of n-grams. In the code below, we will build bigram vocabulary from our news dataset:

In [12]:
from torchtext.data.utils import ngrams_iterator

bi_counter = collections.Counter()
for (label, line) in train_dataset:
    bi_counter.update(ngrams_iterator(tokenizer(line),ngrams=2))
bi_vocab = torchtext.vocab.Vocab(bi_counter, min_freq=2)

print(f"Bigram vocab size = {len(bi_vocab)}")

Bigram vocab size = 481971


In [13]:
def encode(x):
    return [bi_vocab.stoi[s] for s in tokenizer(x)]

encode(first_sentence)

[572,
 564,
 2,
 2326,
 49106,
 150,
 88,
 3,
 1143,
 14,
 32,
 15,
 32,
 16,
 443749,
 4,
 572,
 499,
 17,
 10,
 0,
 7,
 468770,
 4,
 52,
 7019,
 1050,
 442,
 2]

The main drawback of N-gram approach is that vocabulary size starts to grow extremely fast. Here we specify `min_freq` flag to `Vocab` constructor in order to avoid those tokens that appear in the text only once. We can also increase `min_freq` even further, because infrequent words/phrases usually have little effect on the accuracy of classification.

> **Note:** Try setting set `min_freq` parameter to a higher value, and observe the length of vocabulary change.

In practice, n-gram vocabulary size is still too high to represent words as one-hot vectors, and thus we need to combine this representation with some dimensionality reduction techniques, such as *embeddings*, which we will discuss in a later unit.

## Feeding text into neural network

We have learnt how to represent each word by a number. Now, to build text classification model, we need to feed the whole sentence (or whole news article) into a neural network. The problem here is that each article/sentence has variable length; and all fully-connected or convolution neural networks deal with fixed input size. There are two ways we can handle this problem:

* Find a way to collapse a sentence into fixed-length vector. In the next unit we will see how **Bag-of-Words** and **TF-IDF** representations help us to do that.
* Design special neural network architectures that can deal with variable length sequences. We'll learn how Recurrent neural networks (RNN) for sequence modeling are implemented later in this module.